# Numerical Linear Algebra for Machine Learning
## Computational Efficiency and Large-Scale Operations

Welcome to the **computational heart** of machine learning! While theoretical linear algebra tells us what's possible, numerical linear algebra makes it **fast and practical** at scale.

### What You'll Master
By the end of this notebook, you'll understand:
1. **Numerical stability** - Why some algorithms fail in practice
2. **Matrix decompositions** - The building blocks of efficient computation
3. **Iterative methods** - Solving huge systems without storing full matrices
4. **Computational complexity** - Why some operations scale better than others
5. **Memory optimization** - Making algorithms work with limited resources
6. **GPU acceleration** - Parallel computing for massive speedups

### Why This is Critical
- **Modern AI models** have billions of parameters
- **Naive algorithms** would take centuries to train large models
- **Numerical methods** make the impossible practical
- **Understanding efficiency** helps you design scalable systems

### Real-World Impact
- **GPT models**: Trained using optimized matrix operations
- **Recommendation systems**: Handle millions of users efficiently
- **Computer vision**: Process high-resolution images in real-time

Let's make machine learning lightning fast! ⚡

In [ ]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
import seaborn as sns
import time
from scipy.linalg import solve, lstsq, svd, qr, cholesky, lu
from scipy.sparse.linalg import spsolve, cg, gmres
from sklearn.datasets import make_regression, make_classification
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

# For GPU operations (if available)
try:
    import cupy as cp
    GPU_AVAILABLE = True
    print("🚀 GPU acceleration available with CuPy!")
except ImportError:
    GPU_AVAILABLE = False
    print("💻 Using CPU computation (install CuPy for GPU acceleration)")

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
np.random.seed(42)

print("⚡ Numerical Linear Algebra toolkit loaded!")
print("Ready for high-performance computing!")

## 1. Computational Complexity: Understanding the Cost

### Why Complexity Matters
In machine learning, we often deal with:
- **Massive datasets**: Millions of samples
- **High dimensions**: Thousands of features
- **Deep networks**: Billions of parameters

A naive algorithm might work for small problems but **fail completely** at scale!

### Big O Notation Refresher
- **O(n)**: Linear time - doubles when data doubles
- **O(n²)**: Quadratic time - 4x slower when data doubles
- **O(n³)**: Cubic time - 8x slower when data doubles
- **O(log n)**: Logarithmic time - barely slower as data grows

### Common Linear Algebra Operations

| Operation | Naive Complexity | Optimized Complexity | Memory |
|-----------|------------------|---------------------|--------|
| Matrix-Vector | O(n²) | O(n²) | O(n²) |
| Matrix-Matrix | O(n³) | O(n^2.376) | O(n²) |
| Matrix Inverse | O(n³) | O(n³) | O(n²) |
| SVD | O(mn²) | O(mn²) | O(mn) |
| Eigenvalues | O(n³) | O(n³) | O(n²) |

### The Scalability Crisis
For a matrix of size n×n:
- **n = 1,000**: Matrix multiplication takes ~1 second
- **n = 10,000**: Matrix multiplication takes ~17 minutes
- **n = 100,000**: Matrix multiplication takes ~12 days!

**Solution**: Use specialized algorithms and avoid unnecessary operations!

In [ ]:
def benchmark_matrix_operations():
    """Benchmark common matrix operations at different scales"""
    
    sizes = [100, 500, 1000, 2000]
    operations = ['Matrix-Vector', 'Matrix-Matrix', 'Matrix Inverse', 'SVD']
    
    results = {op: [] for op in operations}
    
    print("⏱️  Benchmarking Matrix Operations")
    print("=" * 50)
    print(f"{'Size':<6} {'Mat-Vec':<10} {'Mat-Mat':<10} {'Inverse':<10} {'SVD':<10}")
    print("-" * 50)
    
    for size in sizes:
        # Generate random matrices
        A = np.random.randn(size, size)
        B = np.random.randn(size, size)
        x = np.random.randn(size)
        
        # Matrix-Vector multiplication
        start = time.time()
        _ = A @ x
        mat_vec_time = time.time() - start
        results['Matrix-Vector'].append(mat_vec_time)
        
        # Matrix-Matrix multiplication
        start = time.time()
        _ = A @ B
        mat_mat_time = time.time() - start
        results['Matrix-Matrix'].append(mat_mat_time)
        
        # Matrix inverse (if not too large)
        if size <= 1000:
            start = time.time()
            _ = np.linalg.inv(A)
            inv_time = time.time() - start
            results['Matrix Inverse'].append(inv_time)
        else:
            results['Matrix Inverse'].append(np.nan)
            inv_time = float('inf')
        
        # SVD (if not too large)
        if size <= 1000:
            start = time.time()
            _ = np.linalg.svd(A)
            svd_time = time.time() - start
            results['SVD'].append(svd_time)
        else:
            results['SVD'].append(np.nan)
            svd_time = float('inf')
        
        # Print results
        print(f"{size:<6} {mat_vec_time:<10.4f} {mat_mat_time:<10.4f} ",
              f"{inv_time if inv_time != float('inf') else 'N/A':<10} ",
              f"{svd_time if svd_time != float('inf') else 'N/A':<10}")
    
    # Plot scaling behavior
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.flatten()
    
    for i, (op, times) in enumerate(results.items()):
        valid_sizes = []
        valid_times = []
        
        for j, t in enumerate(times):
            if not np.isnan(t):
                valid_sizes.append(sizes[j])
                valid_times.append(t)
        
        if valid_times:
            axes[i].loglog(valid_sizes, valid_times, 'o-', linewidth=2, markersize=8)
            axes[i].set_xlabel('Matrix Size (n)')
            axes[i].set_ylabel('Time (seconds)')
            axes[i].set_title(f'{op} Scaling')
            axes[i].grid(True, alpha=0.3)
            
            # Add theoretical scaling lines
            if op == 'Matrix-Vector':
                theoretical = [(s/valid_sizes[0])**2 * valid_times[0] for s in valid_sizes]
                axes[i].loglog(valid_sizes, theoretical, '--', alpha=0.7, label='O(n²) theoretical')
            elif op == 'Matrix-Matrix':
                theoretical = [(s/valid_sizes[0])**3 * valid_times[0] for s in valid_sizes]
                axes[i].loglog(valid_sizes, theoretical, '--', alpha=0.7, label='O(n³) theoretical')
            
            axes[i].legend()
    
    plt.tight_layout()
    plt.show()
    
    return results

# Run benchmark
benchmark_results = benchmark_matrix_operations()

## 2. Matrix Decompositions: The Workhorses of Numerical Computing

### Why Decompositions?
Instead of working with the original matrix, we **decompose** it into simpler parts:
- **Easier to compute** with
- **More numerically stable**
- **Reveal mathematical structure**
- **Enable efficient algorithms**

### Key Decompositions in ML

#### 1. LU Decomposition: A = LU
- **L**: Lower triangular matrix
- **U**: Upper triangular matrix
- **Use**: Solving linear systems efficiently

#### 2. QR Decomposition: A = QR
- **Q**: Orthogonal matrix (Q^T Q = I)
- **R**: Upper triangular matrix
- **Use**: Linear regression, least squares

#### 3. SVD: A = UΣV^T
- **U, V**: Orthogonal matrices
- **Σ**: Diagonal matrix (singular values)
- **Use**: Dimensionality reduction, pseudoinverse

#### 4. Eigendecomposition: A = QΛQ^T
- **Q**: Eigenvectors
- **Λ**: Eigenvalues (diagonal)
- **Use**: PCA, spectral methods

#### 5. Cholesky: A = LL^T
- **L**: Lower triangular matrix
- **Requirement**: A must be positive definite
- **Use**: Gaussian processes, covariance matrices

In [ ]:
def demonstrate_matrix_decompositions():
    """Show different matrix decompositions and their properties"""
    
    print("🔧 Matrix Decomposition Showcase")
    print("=" * 40)
    
    # Create a test matrix
    np.random.seed(42)
    n = 5
    A = np.random.randn(n, n)
    A = A + A.T  # Make symmetric for some decompositions
    A = A + n * np.eye(n)  # Make positive definite
    
    print(f"Original matrix A ({n}x{n}):")
    print(A)
    print(f"Condition number: {np.linalg.cond(A):.2f}")
    print()
    
    # 1. LU Decomposition
    print("1. LU Decomposition: A = LU")
    P, L, U = lu(A)
    print(f"   Reconstruction error: {np.linalg.norm(P @ L @ U - A):.2e}")
    print(f"   L is lower triangular: {np.allclose(L, np.tril(L))}")
    print(f"   U is upper triangular: {np.allclose(U, np.triu(U))}")
    print()
    
    # 2. QR Decomposition
    print("2. QR Decomposition: A = QR")
    Q, R = qr(A)
    print(f"   Reconstruction error: {np.linalg.norm(Q @ R - A):.2e}")
    print(f"   Q is orthogonal: {np.linalg.norm(Q.T @ Q - np.eye(n)):.2e}")
    print(f"   R is upper triangular: {np.allclose(R, np.triu(R))}")
    print()
    
    # 3. SVD
    print("3. Singular Value Decomposition: A = UΣV^T")
    U, s, Vt = svd(A)
    print(f"   Reconstruction error: {np.linalg.norm(U @ np.diag(s) @ Vt - A):.2e}")
    print(f"   Singular values: {s}")
    print(f"   Rank: {np.sum(s > 1e-10)}")
    print()
    
    # 4. Eigendecomposition
    print("4. Eigendecomposition: A = QΛQ^T")
    eigenvalues, eigenvectors = np.linalg.eig(A)
    A_reconstructed = eigenvectors @ np.diag(eigenvalues) @ eigenvectors.T
    print(f"   Reconstruction error: {np.linalg.norm(A_reconstructed - A):.2e}")
    print(f"   Eigenvalues: {eigenvalues.real}")
    print(f"   All eigenvalues positive: {np.all(eigenvalues.real > 0)}")
    print()
    
    # 5. Cholesky Decomposition
    print("5. Cholesky Decomposition: A = LL^T")
    try:
        L_chol = cholesky(A, lower=True)
        print(f"   Reconstruction error: {np.linalg.norm(L_chol @ L_chol.T - A):.2e}")
        print(f"   L is lower triangular: {np.allclose(L_chol, np.tril(L_chol))}")
        print(f"   Computational advantage: Only need to store lower triangle")
    except np.linalg.LinAlgError:
        print("   Matrix is not positive definite - Cholesky not applicable")
    print()
    
    return A, (P, L, U), (Q, R), (U, s, Vt), (eigenvalues, eigenvectors)

# Demonstrate decompositions
decomp_results = demonstrate_matrix_decompositions()

## 3. Solving Linear Systems: From Direct to Iterative Methods

### The Fundamental Problem
In machine learning, we constantly solve systems like:
```
Ax = b
```
Where:
- **A**: Coefficient matrix (often large and sparse)
- **x**: Unknown vector (what we want to find)
- **b**: Right-hand side vector (known)

### Method Categories

#### Direct Methods
- **Solve exactly** in finite steps
- **Examples**: Gaussian elimination, LU decomposition
- **Pros**: Exact solution (up to machine precision)
- **Cons**: O(n³) complexity, requires storing full matrix

#### Iterative Methods
- **Approximate solution** through iteration
- **Examples**: Conjugate gradient, GMRES
- **Pros**: Memory efficient, good for sparse matrices
- **Cons**: May not converge, approximate solution

### When to Use What?

| Matrix Type | Size | Best Method | Why |
|-------------|------|-------------|-----|
| Dense, small | < 1000 | Direct (LU) | Fast and exact |
| Dense, large | > 10000 | Iterative | Memory constraints |
| Sparse, any | Any | Iterative | Exploits sparsity |
| Positive definite | Any | Cholesky/CG | Numerical stability |